# RPGLE to Spring Boot Conversion Tool

This notebook demonstrates a three-phase approach to convert AS400 RPGLE code to a Spring Boot Java application using Gemini LLM:

1. **Phase 1 - Parse Code Generation**: Analyze RPGLE code structure
2. **Phase 2 - Business Logic Extraction**: Extract business rules and logic
3. **Phase 3 - Java Code Generation**: Generate Spring Boot application

Each phase builds upon the previous one to produce better results.

## Setup and Import Libraries

In [ ]:
!pip install google-generativeai pandas

In [ ]:
import os
import re
from google.colab import files
import pandas as pd
import json
from typing import Dict, List, Any, Optional, Tuple
import google.generativeai as genai
from IPython.display import Markdown, display

## Configure Gemini API

Enter your Google API key to access the Gemini model.

In [ ]:
# Set up Gemini API
GOOGLE_API_KEY = input("Enter your Google API Key: ")
genai.configure(api_key=GOOGLE_API_KEY)

# Model configuration
model_name = "gemini-1.5-pro"

model = genai.GenerativeModel(
    model_name=model_name,
    generation_config={
        "temperature": 0.2,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 8192,
    }
)

print("API configured successfully!")

## Helper Functions

In [ ]:
def display_parsed_code(parsed_code: Dict[str, Any]):
    """Display parsed RPGLE code in a readable format."""
    display(Markdown("## Parsed RPGLE Code Structure"))
    
    for section, items in parsed_code.items():
        display(Markdown(f"### {section.capitalize()}"))
        if isinstance(items, list) and items:
            if isinstance(items[0], dict):
                df = pd.DataFrame(items)
                display(df)
            else:
                for item in items:
                    display(Markdown(f"- {item}"))
        elif isinstance(items, dict):
            for key, value in items.items():
                display(Markdown(f"- **{key}**: {value}"))
        else:
            display(Markdown(f"{items}"))

def display_business_logic(business_logic: Dict[str, Any]):
    """Display extracted business logic in a readable format."""
    display(Markdown("## Business Logic Analysis"))
    
    for section, content in business_logic.items():
        display(Markdown(f"### {section}"))
        if isinstance(content, list):
            for item in content:
                if isinstance(item, dict):
                    for k, v in item.items():
                        display(Markdown(f"- **{k}**: {v}"))
                else:
                    display(Markdown(f"- {item}"))
        else:
            display(Markdown(content))

def save_spring_boot_project(project_files: Dict[str, str], base_dir: str = "spring_boot_project"):
    """Save generated Spring Boot project files to disk."""
    # Create base directory if it doesn't exist
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    # Save each file
    for file_path, content in project_files.items():
        # Create directory structure if needed
        full_path = os.path.join(base_dir, file_path)
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        
        # Write file content
        with open(full_path, 'w') as f:
            f.write(content)
    
    print(f"Spring Boot project saved to {base_dir}")
    print(f"Generated {len(project_files)} files")
    return base_dir

# Phase 1: Parse RPGLE Code

Upload an RPGLE source file and extract its structure.

In [ ]:
# Upload RPGLE code file
print("Upload your RPGLE source code file:")
uploaded = files.upload()
rpgle_filename = list(uploaded.keys())[0]
rpgle_code = uploaded[rpgle_filename].decode('utf-8')

print(f"\nUploaded file: {rpgle_filename}")
print(f"Code length: {len(rpgle_code)} characters")

In [ ]:
def parse_rpgle_code(rpgle_code: str, model) -> Dict[str, Any]:
    """Parse RPGLE code to extract structured information."""
    prompt = f"""
    You are an expert RPGLE code analyzer with deep knowledge of IBM AS400 systems and RPG programming. 
    Your task is to analyze the following RPGLE code and extract structured information.
    
    Refer to the official IBM documentation at https://www.ibm.com/docs/en/i/7.4.0?topic=rpg-ile-programmers-guide
    to ensure accuracy in your analysis, especially for RPGLE-specific syntax and operations.
    
    Parse the code systematically into these components:
    1. File declarations (input/output files, formats)
    2. Data structures and their components
    3. Variables (standalone fields)
    4. Control flow structures (IF/ELSE, DO loops, SELECT/WHEN)
    5. Subroutines/procedures
    6. RPGLE-specific operations mapped to generic programming concepts
    7. Business rules and calculations
    8. Comments and implicit behaviors
    
    Return your analysis as a structured JSON with these keys:
    - files: list of file declarations with file name, format, and usage (input/output/update)
    - dataStructures: data structures with their fields and types
    - variables: standalone variables with types and purpose
    - controlFlow: control structures and their purpose
    - procedures: identified subroutines and procedures
    - businessRules: calculations and business logic
    - comments: important comments that explain business logic
    
    RPGLE CODE:
    ```
    {rpgle_code}
    ```
    
    Format your response as JSON only, with no preamble or explanation.
    """
    
    response = model.generate_content(prompt)
    result = response.text
    
    # Clean up response to ensure it's valid JSON
    json_str = result.strip()
    if json_str.startswith("```json"):
        json_str = json_str.replace("```json", "", 1)
    if json_str.endswith("```"):
        json_str = json_str.replace("```", "", 1)
    
    try:
        parsed_data = json.loads(json_str.strip())
        return parsed_data
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        print("Raw response:")
        print(json_str)
        raise

In [ ]:
# Execute Phase 1: Parse RPGLE code
print("Parsing RPGLE code...")
parsed_code = parse_rpgle_code(rpgle_code, model)
print("Parsing complete!")

# Display parsed code in a structured format
display_parsed_code(parsed_code)

# Save parsed data for reference
with open('parsed_rpgle.json', 'w') as f:
    json.dump(parsed_code, f, indent=2)
print("Parsed code saved to 'parsed_rpgle.json'")

# Phase 2: Extract Business Logic

Extract business rules and logic from the RPGLE code.

In [ ]:
def extract_business_logic(rpgle_code: str, parsed_code: Dict[str, Any], model) -> Dict[str, Any]:
    """Extract business logic from RPGLE code using the Gemini LLM."""
    # Convert parsed_code to a formatted string for inclusion in prompt
    parsed_code_str = json.dumps(parsed_code, indent=2)
    
    prompt = f"""
    You are a Technical Architect specializing in AS400 systems and RPG to Java migrations. 
    Create a detailed comprehensive business logic analysis and technical documentation for the provided RPGLE code.
    
    Original RPGLE Code:
    ```
    {rpgle_code}
    ```
    
    Parsed Code Structure:
    ```json
    {parsed_code_str}
    ```
    
    Your documentation should follow this structure:
    
    1. Introduction: Brief overview, context, and rationale behind the RPG application.
    
    2. Purpose and Scope: Clearly describe the application's purpose, business processes it supports, and scope of functionalities.
    
    3. Program Structure: Detailed explanation of:
       - Code organization and structure
       - Naming conventions
       - External dependencies (files, databases, systems)
    
    4. Key Components and Functional Overview:
       - Entry points
       - Procedures and subroutines
       - File-handling routines
       - Error handling mechanisms
       - Data transformations
    
    5. Data Structure and Definitions: Comprehensive documentation of:
       - Data structures
       - Record formats
       - Tables, arrays
       - Important data elements
    
    6. Logic and Business Rules: Detailed description of:
       - Implemented business logic
       - Conditions and validations
       - Decision-making pathways
       - Processing flows
    
    7. Dependencies and Integrations:
       - External system dependencies
       - Integrations
       - APIs, databases, file systems
       - Communication protocols
    
    8. Performance Considerations:
       - Performance-critical sections
       - Resource usage
       - Known constraints
    
    9. Migration Recommendations:
       - Potential challenges in migration
       - Risks and mitigation strategies
    
    Return your analysis as a structured JSON with the sections above as keys.
    Each section should be a simple string, not a nested structure. Do not include nested objects or arrays.
    Format your response as JSON only, with no preamble or explanation.
    """
    
    response = model.generate_content(prompt)
    result = response.text
    
    # Clean up response to ensure it's valid JSON
    json_str = result.strip()
    if json_str.startswith("```json"):
        json_str = json_str.replace("```json", "", 1)
    if json_str.endswith("```"):
        json_str = json_str.replace("```", "", 1)
    
    try:
        business_logic = json.loads(json_str.strip())
        return business_logic
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        # Try to fix common JSON issues
        try:
            # Clean up line breaks inside JSON values
            import re
            # Replace unescaped newlines inside JSON string values
            cleaned_json = re.sub(r'("[^"]*?)\\n([^"]*?")', r'\1\\\\n\2', json_str)
            cleaned_json = re.sub(r'("[^"]*?)\n([^"]*?")', r'\1 \2', cleaned_json)
            business_logic = json.loads(cleaned_json)
            return business_logic
        except Exception:
            # If still failing, return a simplified structure
            print("Failed to parse JSON. Creating simplified response structure.")
            content_sections = json_str.split('"')
            simplified_response = {
                "Introduction": "See JSON file for full documentation",
                "Documentation": "The full technical documentation was generated but couldn't be properly formatted as JSON. Please check the business_logic.json file directly.",
                "Raw_Content": json_str[:1000] + "... (truncated)"
            }
            return simplified_response

In [ ]:
# Execute Phase 2: Extract business logic
print("Extracting business logic...")
business_logic = extract_business_logic(rpgle_code, parsed_code, model)
print("Business logic extraction complete!")

# Display business logic in a structured format
display_business_logic(business_logic)

# Save business logic to file
print("Saving business logic documentation...")
with open('business_logic.json', 'w') as f:
    # Handle both properly formatted JSON and raw content
    if isinstance(business_logic, dict):
        json.dump(business_logic, f, indent=2)
    else:
        f.write(str(business_logic))
print("Business logic saved to 'business_logic.json'")

# Install FPDF library if needed
try:
    import fpdf
except ImportError:
    print("Installing FPDF library...")
    !pip install fpdf
    import fpdf

# Create PDF with business logic
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'RPGLE Technical Documentation', 0, 1, 'C')
        self.ln(5)
            
pdf = PDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# Add title and file info
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, f"Technical Documentation for: {rpgle_filename}", 0, 1)
pdf.ln(5)

# Add content to PDF - handle any structure
for section, content in business_logic.items():
    # Section header
    pdf.set_font('Arial', 'B', 12)
    pdf.set_fill_color(240, 240, 240)
    pdf.cell(0, 10, f"{section}", 1, 1, 'L', True)
    pdf.set_font('Arial', '', 10)
    
    # Handle different content types
    if isinstance(content, dict):
        for k, v in content.items():
            pdf.set_font('Arial', 'B', 10)
            pdf.cell(0, 8, str(k), 0, 1)
            pdf.set_font('Arial', '', 10)
            pdf.multi_cell(0, 5, str(v))
    elif isinstance(content, list):
        for item in content:
            if isinstance(item, dict):
                for k, v in item.items():
                    pdf.set_font('Arial', 'B', 10)
                    pdf.cell(0, 8, str(k), 0, 1)
                    pdf.set_font('Arial', '', 10)
                    pdf.multi_cell(0, 5, str(v))
            else:
                pdf.multi_cell(0, 5, f"• {str(item)}")
    else:
        # Handle as string (most common case)
        content_str = str(content)
        # Format paragraphs with better spacing
        paragraphs = content_str.split('\n')
        for para in paragraphs:
            if para.strip():
                pdf.multi_cell(0, 5, para)
                pdf.ln(2)
    
    pdf.ln(5)

# Save PDF
pdf_file = 'RPGLE_Technical_Documentation.pdf'
pdf.output(pdf_file)
files.download(pdf_file)
print(f"Technical documentation PDF created and ready for download: {pdf_file}")

# Phase 3: Generate Spring Boot Project

Generate a complete Spring Boot application based on the RPGLE code and extracted business logic.

In [ ]:
def generate_spring_boot_project(rpgle_code: str, parsed_code: Dict[str, Any], 
                              business_logic: Dict[str, Any], model) -> Dict[str, str]:
    """Generate a Spring Boot Java project based on the RPGLE code, parsed structure, and business logic."""
    # Convert parsed_code and business_logic to formatted strings
    parsed_code_str = json.dumps(parsed_code, indent=2)
    business_logic_str = json.dumps(business_logic, indent=2)
    
    prompt = f"""
    You are a Java modernization expert specializing in AS400 to Spring Boot migrations.
    Create a complete Spring Boot application that implements the functionality of the given RPGLE program.
    
    Original RPGLE Code:
    ```
    {rpgle_code}
    ```
    
    Parsed Code Structure:
    ```json
    {parsed_code_str}
    ```
    
    Business Logic Analysis:
    ```json
    {business_logic_str}
    ```
    
    Generate a complete Spring Boot project with the following components:
    
    1. Project structure with proper package organization
    2. Entity classes for data structures
    3. Service classes implementing the business logic
    4. Repository interfaces for data access
    5. REST controllers for API endpoints
    6. Configuration classes
    7. Exception handling
    8. Unit tests for key components
    
    Follow these best practices:
    - Use modern Java features (streams, optionals, etc.)
    - Implement dependency injection
    - Follow RESTful API design principles
    - Include proper error handling and logging
    - Use bean validation for input validation
    - Include comments explaining the business logic
    
    Return a JSON object where each key is a file path (relative to project root) and the value is the file content.
    Key files should include:
    - pom.xml
    - Entity classes
    - Service implementation
    - Repository interfaces
    - Controller classes
    - Any required configuration files
    
    Format your response as JSON only, with no preamble or explanation.
    """
    
    response = model.generate_content(prompt)
    result = response.text
    
    # Clean up response to ensure it's valid JSON
    json_str = result.strip()
    if json_str.startswith("```json"):
        json_str = json_str.replace("```json", "", 1)
    if json_str.endswith("```"):
        json_str = json_str.replace("```", "", 1)
    
    try:
        project_files = json.loads(json_str.strip())
        return project_files
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        print("Raw response:")
        print(json_str[:500] + "..." if len(json_str) > 500 else json_str)
        raise

In [ ]:
# Execute Phase 3: Generate Spring Boot project
print("Generating Spring Boot project...")
project_files = generate_spring_boot_project(rpgle_code, parsed_code, business_logic, model)
print("Spring Boot project generation complete!")

# Display summary of generated files
print(f"Generated {len(project_files)} files:")
for path in sorted(project_files.keys()):
    print(f"- {path}")

# Create project directory structure
project_dir = "spring_boot_project"
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

# Save each file to appropriate location
for file_path, content in project_files.items():
    # Create directory structure if needed
    full_path = os.path.join(project_dir, file_path)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    
    # Write file content
    with open(full_path, 'w') as f:
        f.write(content)

print(f"Spring Boot project saved to {project_dir}")

# Create a zip file for download
import shutil
zip_file = "spring_boot_project.zip"
shutil.make_archive("spring_boot_project", 'zip', project_dir)
files.download(zip_file)
print(f"Download {zip_file} to get the complete project")

# Create a README in the project folder
readme_content = f"""# Generated Spring Boot Project

This project was automatically generated from RPGLE code using Gemini LLM.

## Project Structure

The project contains {len(project_files)} files organized in a standard Spring Boot structure:

- Entity classes
- Repository interfaces 
- Service implementations
- Controllers
- Configuration

## How to run

1. Import as a Maven project in your IDE
2. Run the main Application class
3. Access the API at http://localhost:8080

## Generated from

Original RPGLE file: {rpgle_filename}
"""

with open(os.path.join(project_dir, "README.md"), "w") as f:
    f.write(readme_content)

print("Project README created")

## Sample RPGLE Code (For Testing)

If you don't have an RPGLE file to test with, you can use this sample. Copy it to a text file and upload.

In [ ]:
sample_rpgle = """
     H DATEDIT(*YMD) DATFMT(*ISO) DECEDIT('0.') OPTION(*SRCSTMT)
     F CUSTMAST  IF   E           K DISK    PREFIX(CM_)
     F ORDHDR    IF   E           K DISK    PREFIX(OH_)
     F ORDLINE   IF   E           K DISK    PREFIX(OL_)
     F REPORT    O    E             PRINTER
     
     D OrderTotal      S             11P 2
     D LineTotal       S             11P 2
     D GrandTotal      S             13P 2
     D ReportDate      S               D   DATFMT(*ISO)
     D CustomerFound   S               N
     
     D OrderAnalysis   DS                  QUALIFIED
     D  CustomerId                   10A
     D  CustomerName                 30A
     D  OrderCount                    5P 0
     D  TotalAmount                  13P 2
     D  LastOrderDate                  D   DATFMT(*ISO)
     
     * Main procedure
     C     *ENTRY        PLIST
     C                   PARM                    ReportDate
     
     * Initialize variables
     C                   EVAL      GrandTotal = 0
     
     * Write report header
     C                   WRITE     REPHEADER
     C                   WRITE     COLHEADER
     
     * Process each customer
     C     CM_CUSTNO     SETLL     CUSTMAST
     C                   READ      CUSTMAST
     C                   DOW       NOT %EOF(CUSTMAST)
     
     * Initialize customer analysis
     C                   EVAL      OrderAnalysis.CustomerId = CM_CUSTNO
     C                   EVAL      OrderAnalysis.CustomerName = CM_NAME
     C                   EVAL      OrderAnalysis.OrderCount = 0
     C                   EVAL      OrderAnalysis.TotalAmount = 0
     C                   EVAL      OrderAnalysis.LastOrderDate = D'0001-01-01'
     
     * Find all orders for this customer
     C     CM_CUSTNO     SETLL     ORDHDR
     C     CM_CUSTNO     READE     ORDHDR
     C                   DOW       NOT %EOF(ORDHDR)
     
     * Check if order date is within reporting period
     C                   IF        OH_DATE <= ReportDate
     C                   EVAL      OrderAnalysis.OrderCount += 1
     
     * Update last order date if more recent
     C                   IF        OH_DATE > OrderAnalysis.LastOrderDate
     C                   EVAL      OrderAnalysis.LastOrderDate = OH_DATE
     C                   ENDIF
     
     * Calculate order total
     C                   EVAL      OrderTotal = 0
     
     C     OH_ORDNO      SETLL     ORDLINE
     C     OH_ORDNO      READE     ORDLINE
     C                   DOW       NOT %EOF(ORDLINE)
     
     * Apply business pricing rules
     C                   EVAL      LineTotal = OL_QTY * OL_PRICE
     
     * Apply volume discount if quantity > 10
     C                   IF        OL_QTY > 10
     C                   EVAL      LineTotal = LineTotal * 0.95
     C                   ENDIF
     
     * Apply promotional discount for specific product category
     C                   IF        OL_CATEGORY = 'PROMO'
     C                   EVAL      LineTotal = LineTotal * 0.90
     C                   ENDIF
     
     C                   EVAL      OrderTotal = OrderTotal + LineTotal
     
     C     OH_ORDNO      READE     ORDLINE
     C                   ENDDO
     
     * Add shipping fee for small orders
     C                   IF        OrderTotal < 100
     C                   EVAL      OrderTotal = OrderTotal + 10.00
     C                   ENDIF
     
     C                   EVAL      OrderAnalysis.TotalAmount += OrderTotal
     
     C                   ENDIF
     
     C     CM_CUSTNO     READE     ORDHDR
     C                   ENDDO
     
     * Write customer summary if there were orders
     C                   IF        OrderAnalysis.OrderCount > 0
     C                   WRITE     CUSTLINE
     C                   EVAL      GrandTotal = GrandTotal + 
     C                                OrderAnalysis.TotalAmount
     C                   ENDIF
     
     C     CM_CUSTNO     READ      CUSTMAST
     C                   ENDDO
     
     * Write report footer
     C                   WRITE     TOTLINE
     
     C                   EVAL      *INLR = *ON
     C                   RETURN
     
     * Report formats
     OREPHEADER   E
     O                                           'CUSTOMER ORDER REPORT'
     O                                           'RUN DATE: '
     O                       ReportDate
     
     OCOLHEADER   E
     O                                           'CUSTOMER ID'
     O                                           'CUSTOMER NAME'
     O                                           'ORDER COUNT'
     O                                           'TOTAL AMOUNT'
     O                                           'LAST ORDER'
     
     OCUSTLINE    E
     O                       OrderAnalysis.CustomerId
     O                       OrderAnalysis.CustomerName
     O                       OrderAnalysis.OrderCount
     O                       OrderAnalysis.TotalAmount
     O                       OrderAnalysis.LastOrderDate
     
     OTOTLINE     E
     O                                           'GRAND TOTAL:'
     O                       GrandTotal
"""

# Save the sample code to a file for upload
sample_file = "sample_rpgle_code.rpgle"
with open(sample_file, 'w') as f:
    f.write(sample_rpgle)
    
print(f"Sample RPGLE code saved to {sample_file}")
print("You can upload this file in the Phase 1 step to test the conversion process.")